In [6]:
# @title hitemanAPI.ipynb - Setup API dan Eksekusi Model TTS

# --- Instalasi Library yang dibutuhkan khusus untuk API  ---
!pip install fastapi uvicorn pyngrok nest_asyncio pydantic -q

import os
import asyncio
import nest_asyncio # Diperlukan untuk Uvicorn di Colab
from pyngrok import ngrok # Untuk mengekspos API dari Colab
import uvicorn # Server ASGI untuk FastAPI
import datetime

from fastapi import FastAPI, HTTPException
from fastapi.responses import FileResponse # Untuk mengirim file audio
from pydantic import BaseModel # Untuk validasi data request

print("--- hitemanAPI - Setup API dan Eksekusi Model TTS ---")

# --- Bagian untuk menjalankan notebook hitemanTTS.ipynb ---
# Ini akan membuat semua fungsi dan variabel dari hitemanTTS.ipynb tersedia di sini.

# 1. Mount Google Drive Anda
from google.colab import drive
try:
    drive.mount('/content/drive', force_remount=True) # force_remount jika sesi sebelumnya tidak unmount dengan benar
    print("Info (hitemanAPI): Google Drive berhasil di-mount.")
except Exception as e_mount:
    print(f"ERROR (hitemanAPI): Gagal me-mount Google Drive - {e_mount}")
    raise # Hentikan jika Drive tidak bisa di-mount

# 2. Tentukan PATH ke notebook hitemanTTS.ipynb di Google Drive Anda
# !!! PENTING: SESUAIKAN PATH INI DENGAN LOKASI NOTEBOOK ANDA !!!
PATH_TO_HITEMAN_TTS_NOTEBOOK = '/content/drive/My Drive/Colab Notebooks/hitemanTTS.ipynb'
# Contoh lain:
# PATH_TO_HITEMAN_TTS_NOTEBOOK = '/content/drive/My Drive/ProyekAI/hitemanTTS.ipynb'

print(f"Info (hitemanAPI): Akan mencoba menjalankan notebook layanan dari: {PATH_TO_HITEMAN_TTS_NOTEBOOK}")

# 3. Jalankan notebook hitemanTTS.ipynb menggunakan magic command %run
# Tanda kutip ("$PATH_...") penting jika path Anda mengandung spasi.
try:
    if os.path.exists(PATH_TO_HITEMAN_TTS_NOTEBOOK):
        %run "$PATH_TO_HITEMAN_TTS_NOTEBOOK"
        print("Info (hitemanAPI): Notebook layanan (hitemanTTS.ipynb) berhasil dijalankan.")

        # Verifikasi singkat bahwa beberapa elemen penting dari hitemanTTS tersedia
        if 'GEMINI_API_KEY' in globals() and GEMINI_API_KEY: # GEMINI_API_KEY dari hitemanTTS
            print(f"  Verifikasi: GEMINI_API_KEY dari hitemanTTS terdeteksi: {GEMINI_API_KEY[:10]}...")
        else:
            print("  PERINGATAN Verifikasi: GEMINI_API_KEY dari hitemanTTS TIDAK terdeteksi.")

        if 'generate_therapy_script' in globals():
            print("  Verifikasi: Fungsi 'generate_therapy_script' dari hitemanTTS terdeteksi.")
        else:
            print("  PERINGATAN Verifikasi: Fungsi 'generate_therapy_script' dari hitemanTTS TIDAK terdeteksi.")

        if 'generate_audio_with_native_tts' in globals():
            print("  Verifikasi: Fungsi 'generate_audio_with_native_tts' dari hitemanTTS terdeteksi.")
        else:
            print("  PERINGATAN Verifikasi: Fungsi 'generate_audio_with_native_tts' dari hitemanTTS TIDAK terdeteksi.")

        if 'AUDIO_OUTPUT_DIR_TTS' in globals():
            print(f"  Verifikasi: AUDIO_OUTPUT_DIR_TTS dari hitemanTTS terdeteksi: {AUDIO_OUTPUT_DIR_TTS}")
        else:
             print("  PERINGATAN Verifikasi: AUDIO_OUTPUT_DIR_TTS dari hitemanTTS TIDAK terdeteksi.")


    else:
        print(f"ERROR (hitemanAPI): File notebook layanan '{PATH_TO_HITEMAN_TTS_NOTEBOOK}' tidak ditemukan. Periksa kembali path Anda.")
        raise FileNotFoundError(f"Notebook layanan tidak ditemukan di {PATH_TO_HITEMAN_TTS_NOTEBOOK}")

except Exception as e_run_services:
    print(f"ERROR (hitemanAPI): Terjadi kesalahan saat menjalankan notebook layanan '{PATH_TO_HITEMAN_TTS_NOTEBOOK}' - {e_run_services}")
    print("Pastikan tidak ada error di notebook hitemanTTS.ipynb saat dijalankan sendiri.")
    raise # Hentikan jika ada error saat %run

print("\n--- Selesai: hitemanAPI - Sel 1 ---")

--- hitemanAPI - Setup API dan Eksekusi Model TTS ---
Mounted at /content/drive
Info (hitemanAPI): Google Drive berhasil di-mount.
Info (hitemanAPI): Akan mencoba menjalankan notebook layanan dari: /content/drive/My Drive/Colab Notebooks/hitemanTTS.ipynb
--- hitemanTTS - Sel 1: Setup dan Konfigurasi ---
Info (hitemanTTS): API Key Gemini berhasil diambil dari Colab Secrets.
Info (hitemanTTS): Konfigurasi global google.generativeai dengan API Key berhasil.
Info (hitemanTTS): Direktori output audio diatur ke: temp_hiteman_audio_from_tts_notebook

--- Selesai: hitemanTTS ---
--- hitemanTTS - Layanan Model Gemini ---
Info (hitemanTTS): Akan menggunakan model TTS Native: models/gemini-2.5-flash-preview-tts
Info (hitemanTTS): Model teks Gemini (models/gemini-1.5-flash) berhasil diinisialisasi.
--- Selesai: hitemanTTS - Sel 2 ---
Info (hitemanAPI): Notebook layanan (hitemanTTS.ipynb) berhasil dijalankan.
  Verifikasi: GEMINI_API_KEY dari hitemanTTS terdeteksi: AIzaSyBQap...
  Verifikasi: Fungs

In [7]:
# @title hitemanAPI.ipynb -  Aplikasi FastAPI

print("--- hitemanAPI - Aplikasi FastAPI ---")

# Pydantic model untuk validasi data request body
class EmotionInputModel(BaseModel):
    emotion: str # Contoh input: "sedih", "cemas", "khawatir"

# Inisialisasi aplikasi FastAPI
# Gunakan nama 'app' karena ini yang umum dicari oleh Uvicorn
app = FastAPI(
    title="HiTeman API - Terapi Audio dengan Native Gemini TTS",
    description="API ini menerima label emosi (sedih, cemas, khawatir), menghasilkan skrip terapi yang dipersonalisasi menggunakan model bahasa Gemini, lalu mengubah skrip tersebut menjadi audio (.wav) menggunakan kemampuan TTS native Gemini.",
    version="2.1.0", # Versi API
    contact={
        "name": "HiTemanDev Team",
        "url": "https://github.com/HiTemanDev", # Ganti dengan URL relevan
    }
)

# Event handler yang akan dijalankan saat aplikasi FastAPI startup
@app.on_event("startup")
async def handle_fastapi_startup_event():
    """
    Fungsi ini dipanggil sekali saat aplikasi FastAPI dimulai.
    Digunakan untuk melakukan inisialisasi atau setup yang diperlukan.
    """
    print("Info (hitemanAPI - Startup): Memulai proses startup HiTeman API...")

    # Pastikan model teks Gemini dari notebook layanan sudah siap.
    # Fungsi initialize_gemini_language_model() dan variabel gemini_text_model_is_ready
    # seharusnya sudah tersedia karena %run dari hitemanTTS.ipynb
    if 'initialize_gemini_language_model' in globals():
        if not initialize_gemini_language_model(): # Panggil fungsi dari hitemanTTS
            print("ERROR (hitemanAPI - Startup): Gagal menginisialisasi model teks Gemini saat startup API. Endpoint terapi mungkin tidak berfungsi dengan benar.")
        # else: # Tidak perlu print sukses lagi karena sudah ada di dalam fungsi init

    if 'GEMINI_API_KEY' not in globals() or not GEMINI_API_KEY: # Cek variabel dari hitemanTTS
         print(f"PERINGATAN (hitemanAPI - Startup): GEMINI_API_KEY tidak terdeteksi. Semua fitur yang bergantung pada Gemini akan gagal.")

    print("Info (hitemanAPI - Startup): Proses startup HiTeman API selesai. Aplikasi siap menerima permintaan.")

# Endpoint root untuk memeriksa status dasar API
@app.get("/",
         summary="Status Dasar API HiTeman",
         tags=["General"],
         response_description="Pesan selamat datang dan status operasional API.")
async def get_api_root_status_check(): # Tidak perlu async jika tidak ada await di dalamnya
    """Endpoint ini memberikan status dasar bahwa API HiTeman berjalan dan siap melayani."""

    # Pastikan DEFAULT_NATIVE_GEMINI_TTS_MODEL_NAME tersedia dari hitemanTTS
    tts_model_name_display = "Tidak Diketahui (Setup Error)"
    if 'DEFAULT_NATIVE_GEMINI_TTS_MODEL_NAME' in globals():
        tts_model_name_display = DEFAULT_NATIVE_GEMINI_TTS_MODEL_NAME

    status_detail = {
        "message": "Selamat datang di HiTeman Terapi Audio API!",
        "status": "operational",
        "tts_model_in_use": tts_model_name_display,
        "timestamp": datetime.datetime.now(tz=datetime.timezone.utc).isoformat() # Ambil timestamp secara langsung
    }
    return status_detail

# Endpoint utama untuk generasi audio terapi
@app.post("/generate_therapy_audio",
            summary="Generasi Audio Terapi dari Label Emosi",
            response_description="File audio WAV yang berisi panduan terapi audio. Akan diunduh oleh User.",
            tags=["Therapy Audio Generation"])
async def api_endpoint_create_therapy_audio_from_emotion(input_data: EmotionInputModel):
    """
    Menerima label emosi (misalnya, "happy", "sadness", "fear", "stress").
    1. Menghasilkan skrip terapi yang dipersonalisasi menggunakan model bahasa Gemini.
    2. Mengubah skrip tersebut menjadi file audio (.wav) menggunakan kemampuan TTS native Gemini.
    3. Mengembalikan file audio tersebut kepada user.
    """
    # Periksa kesiapan layanan Gemini (fungsi dan variabel dari hitemanTTS.ipynb)
    if 'gemini_text_model_is_ready' not in globals() or not gemini_text_model_is_ready:
        # Coba inisialisasi lagi jika startup gagal atau flag tidak diset
        if 'initialize_gemini_language_model' in globals() and not initialize_gemini_language_model():
             raise HTTPException(status_code=503, detail="Layanan generasi skrip terapi sedang tidak tersedia. Silakan coba lagi nanti.")

    if 'GEMINI_API_KEY' not in globals() or not GEMINI_API_KEY:
        raise HTTPException(status_code=503, detail="Layanan TTS tidak dapat diakses karena konfigurasi API Key bermasalah.")

    # Ambil emosi dari input dan normalisasi (lowercase)
    user_emotion_label = input_data.emotion.lower()

    # Daftar emosi yang secara eksplisit didukung oleh prompt terapi kita
    # Daftar emosi bisa diperluas atau membuatnya lebih dinamis
    explicitly_supported_emotions = ["happy", "sadness", "fear", "stress"]
    if user_emotion_label not in explicitly_supported_emotions:
        print(f"Warning (hitemanAPI): Emosi input '{user_emotion_label}' tidak ada dalam daftar utama ({explicitly_supported_emotions}), namun tetap akan dicoba proses oleh Gemini.")
        # Jika ingin lebih ketat, bisa kembalikan error 400:
        # raise HTTPException(status_code=400, detail=f"Emosi '{user_emotion_label}' tidak secara eksplisit didukung untuk skrip terapi saat ini. Emosi yang didukung: {', '.join(explicitly_supported_emotions)}.")

    try:
        # Langkah 1: Generate skrip terapi dalam bentuk teks
        print(f"Info (hitemanAPI): Meminta skrip terapi untuk emosi: '{user_emotion_label}'...")
        # Panggil fungsi dari hitemanTTS.ipynb
        therapy_script_content = await generate_therapy_script(user_emotion_label)
        print(f"Info (hitemanAPI): Skrip terapi berhasil dihasilkan: '{therapy_script_content[:120]}...'") # Log sebagian skrip

        # Langkah 2: Generate audio dari skrip teks menggunakan Native Gemini TTS
        print(f"Info (hitemanAPI): Meminta generasi audio TTS untuk skrip yang dihasilkan...")
        # Panggil fungsi dari hitemanTTS.ipynb
        output_audio_filepath = await generate_audio_with_native_tts(therapy_script_content)
        print(f"Info (hitemanAPI): File audio WAV terapi berhasil dibuat di path: {output_audio_filepath}")

        # Pastikan file audio benar-benar ada sebelum mengirim respons
        if not os.path.exists(output_audio_filepath):
            error_message = f"File audio tidak ditemukan di path yang diharapkan ({output_audio_filepath}) setelah proses TTS."
            print(f"ERROR (hitemanAPI): {error_message}")
            raise HTTPException(status_code=500, detail=f"Terjadi kesalahan internal server: {error_message}")

        # Kirim file audio .wav sebagai respons ke klien
        return FileResponse(
            path=output_audio_filepath,
            media_type='audio/wav', # Media type harus sesuai dengan format file (WAV)
            filename=os.path.basename(output_audio_filepath) # Memberi nama file saat diunduh oleh klien
        )

    except RuntimeError as e_runtime_service: # Menangkap error yang di-raise dari modul layanan
        # Ini biasanya error dari proses di Gemini (teks atau TTS)
        print(f"Error Runtime dari Layanan (hitemanAPI): {e_runtime_service}")
        raise HTTPException(status_code=500, detail=f"Terjadi kesalahan internal pada server saat memproses permintaan audio terapi: {str(e_runtime_service)}")
    except Exception as e_general_api: # Menangkap error tak terduga lainnya di level API
        print(f"Error Tak Terduga di Endpoint /generate_therapy_audio (hitemanAPI): {e_general_api}")
        raise HTTPException(status_code=500, detail="Terjadi kesalahan internal server yang tidak terduga.")

print("--- Selesai: hitemanAPI - Sel 2 ---")

--- hitemanAPI - Aplikasi FastAPI ---
--- Selesai: hitemanAPI - Sel 2 ---


<ipython-input-7-b71857504ecd>:22: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [8]:
# @title hitemanAPI.ipynb : Skrip untuk Menjalankan Server FastAPI di Colab (DENGAN NGROK AUTH)

print("--- hitemanAPI - Skrip untuk Menjalankan Server FastAPI di Colab ---")

# Variabel global untuk menyimpan URL publik Ngrok jika server berjalan
ngrok_public_url_global_api = None

async def run_hiteman_fastapi_server_on_colab_final(): # Nama fungsi dari sebelumnya
    """
    Fungsi utama untuk menjalankan server FastAPI menggunakan Uvicorn
    dan mengeksposnya ke publik melalui Ngrok saat berjalan di Google Colab.
    """
    global ngrok_public_url_global_api # Menggunakan variabel global

    nest_asyncio.apply() # Diperlukan agar Uvicorn bisa berjalan dengan baik di event loop Colab

    # --- Mengambil dan Mengkonfigurasi Ngrok Authtoken dari Colab Secrets ---
    NGROK_COLAB_AUTHTOKEN = None
    try:
        from google.colab import userdata
        NGROK_COLAB_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN') # Ganti 'NGROK_AUTHTOKEN' jika Anda menggunakan nama secret lain
        if NGROK_COLAB_AUTHTOKEN:
            try:
                ngrok.set_auth_token(NGROK_COLAB_AUTHTOKEN)
                print("Info (hitemanAPI): Authtoken Ngrok berhasil dikonfigurasi dari Colab Secrets.")
            except Exception as e_ngrok_auth:
                print(f"Warning (hitemanAPI): Gagal set Ngrok authtoken - {e_ngrok_auth}")
                print("  Pastikan authtoken yang Anda masukkan ke Colab Secrets valid.")
        else:
            print("PERINGATAN (hitemanAPI): Secret 'NGROK_AUTHTOKEN' tidak ditemukan atau kosong di Colab Secrets.")
            print("  Ngrok akan gagal tanpa authtoken. Silakan buat akun Ngrok dan tambahkan authtoken ke Secrets.")
            print("  Link: https://dashboard.ngrok.com/get-started/your-authtoken")
            return # Keluar jika tidak ada authtoken, karena Ngrok akan gagal
    except ImportError:
        print("Warning (hitemanAPI): Modul userdata tidak ditemukan (bukan di Colab?). Ngrok mungkin memerlukan konfigurasi manual.")
    except Exception as e_secret:
        print(f"Error (hitemanAPI): Terjadi kesalahan saat mengambil Ngrok authtoken dari Secrets: {e_secret}")
        return

    # Upaya untuk menutup tunnel Ngrok yang mungkin sudah ada dari sesi sebelumnya
    try:
        active_ngrok_tunnels = ngrok.get_tunnels()
        for tunnel_obj in active_ngrok_tunnels:
            ngrok.disconnect(tunnel_obj.public_url)
            print(f"Info (hitemanAPI): Tunnel Ngrok lama di {tunnel_obj.public_url} berhasil ditutup.")
    except Exception:
        pass # Abaikan jika tidak ada tunnel aktif atau error saat disconnect

    # Mulai tunnel Ngrok baru.
    try:
        print("Info (hitemanAPI): Mencoba membuat tunnel Ngrok...")
        ngrok_public_tunnel = ngrok.connect(8000) # Uvicorn akan berjalan di port 8000
        ngrok_public_url_global_api = ngrok_public_tunnel.public_url
        print(f"\n SERVER API HITEMAN BERHASIL DIMULAI!")
        print(f"====================================================================")
        print(f"  URL Publik (Ngrok)      : {ngrok_public_url_global_api}")
        print(f"  Dokumentasi Interaktif  : {ngrok_public_url_global_api}/docs (Swagger UI)")
        print(f"  Dokumentasi Alternatif  : {ngrok_public_url_global_api}/redoc")
        print(f"====================================================================")
        print(f"  (Server akan terus berjalan. Untuk menghentikan, stop eksekusi sel ini di Colab.)")
    except Exception as e_ngrok_connect_error:
        print(f"ERROR (hitemanAPI): Gagal membuat tunnel Ngrok - {e_ngrok_connect_error}")
        print("  Pastikan authtoken Ngrok Anda valid dan sudah dikonfigurasi melalui Colab Secrets.")
        print("  Cek juga koneksi internet Anda.")
        return # Keluar dari fungsi jika Ngrok gagal

    # Konfigurasi dan jalankan server Uvicorn untuk aplikasi FastAPI kita
    uvicorn_server_config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    fastapi_server = uvicorn.Server(uvicorn_server_config)

    try:
        await fastapi_server.serve()
    except (KeyboardInterrupt, SystemExit):
        print("\nInfo (hitemanAPI): Server FastAPI dihentikan.")
    finally:
        print("Info (hitemanAPI): Melakukan cleanup dan menutup tunnel Ngrok...")
        if ngrok_public_url_global_api:
            try: ngrok.disconnect(ngrok_public_url_global_api)
            except: pass
        try: ngrok.kill()
        except: pass
        print("Info (hitemanAPI): Server HiTeman telah berhenti sepenuhnya.")

# (Bagian catatan cara menjalankan bisa tetap sama)
print("--- Selesai: hitemanAPI ----")

--- hitemanAPI - Skrip untuk Menjalankan Server FastAPI di Colab ---
--- Selesai: hitemanAPI ----


In [9]:
# @title hitemanAPI.ipynb - AKTIFKAN SERVER API HITEMAN
import asyncio
# Fungsi 'run_hiteman_fastapi_server_on_colab_final' dari Sel 3.
# Objek 'app' dari Sel 2.
# Fungsi layanan dari hitemanTTS.ipynb (via %run di Sel 1).

print("Mencoba menjalankan server FastAPI HiTeman...")
await run_hiteman_fastapi_server_on_colab_final()
print("Eksekusi sel server telah selesai atau dihentikan.")

Mencoba menjalankan server FastAPI HiTeman...
Info (hitemanAPI): Authtoken Ngrok berhasil dikonfigurasi dari Colab Secrets.
Info (hitemanAPI): Mencoba membuat tunnel Ngrok...

 SERVER API HITEMAN BERHASIL DIMULAI!
  URL Publik (Ngrok)      : https://3f9f-35-201-146-89.ngrok-free.app
  Dokumentasi Interaktif  : https://3f9f-35-201-146-89.ngrok-free.app/docs (Swagger UI)
  Dokumentasi Alternatif  : https://3f9f-35-201-146-89.ngrok-free.app/redoc
  (Server akan terus berjalan. Untuk menghentikan, stop eksekusi sel ini di Colab.)


INFO:     Started server process [227]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Info (hitemanAPI - Startup): Memulai proses startup HiTeman API...
Info (hitemanAPI - Startup): Proses startup HiTeman API selesai. Aplikasi siap menerima permintaan.
INFO:     116.254.119.158:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     116.254.119.158:0 - "GET /openapi.json HTTP/1.1" 200 OK
Info (hitemanAPI): Meminta skrip terapi untuk emosi: 'stress'...
Info (hitemanAPI): Skrip terapi berhasil dihasilkan: 'Hai... aku dengar kamu lagi merasa tertekan dan kewalahan ya?  Rasanya pasti berat sekali menanggung semua tuntutan itu....'
Info (hitemanAPI): Meminta generasi audio TTS untuk skrip yang dihasilkan...
Info (hitemanTTS): Mengirim TTS ke model 'models/gemini-2.5-flash-preview-tts' dengan config dict: {'response_modalities': ['AUDIO']}
Info (hitemanTTS): Audio berhasil disimpan sebagai file WAV di: temp_hiteman_audio_from_tts_notebook/hiteman_therapy_audio_native_642704153.wav
Info (hitemanAPI): File audio WAV terapi berhasil dibuat di path: temp_hiteman_audio_from_tts_notebook/hiteman

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [227]


Info (hitemanAPI): Melakukan cleanup dan menutup tunnel Ngrok...
Info (hitemanAPI): Server HiTeman telah berhenti sepenuhnya.
Eksekusi sel server telah selesai atau dihentikan.
